In [1]:
# !Rscript ../utils/install_r_libraries.r

In [2]:
%load_ext rpy2.ipython

In [3]:
import os
from datapaths import *

In [4]:
%%R
library(GenomicFeatures)
library(ChIPpeakAnno)
library(ChIPseeker)
library(clusterProfiler)
library(dplyr)
library(stringr)
library(regioneR)

R[write to console]: Loading required package: BiocGenerics

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


R[write to console]: Loading required package: S4Vectors

R[write to console]: Loading required package: stats4

R[write to console]: 
Attaching package: ‘S4Vectors’


R[write to console]: The following objects are masked from ‘package:base’:

    expand.grid, I, unname


R[write to console]: Lo

In [5]:
%%R -i F_GENCODE_STR
txdb_all_annotation <- makeTxDbFromGFF(F_GENCODE_STR, format='gtf')
txdb_all_annotation

R[write to console]: Import genomic features from the file as a GRanges object ... 
R[write to console]: OK

R[write to console]: Prepare the 'metadata' data frame ... 
R[write to console]: OK

R[write to console]: Make the TxDb object ... 
R[write to console]: OK



TxDb object:
# Db type: TxDb
# Supporting package: GenomicFeatures
# Data source: /home/fpavlov/projects/article_conserved_miRNA/data/genome/gencode.vM25.basic.annotation.gtf.gz
# Organism: NA
# Taxonomy ID: NA
# miRBase build ID: NA
# Genome: NA
# Nb of transcripts: 81540
# Db created by: GenomicFeatures package from Bioconductor
# Creation time: 2022-12-21 01:31:30 +0300 (Wed, 21 Dec 2022)
# GenomicFeatures version at creation time: 1.50.2
# RSQLite version at creation time: 2.2.18
# DBSCHEMAVERSION: 1.2


In [6]:
%%R
# https://www.biostars.org/p/140471/
gtf.file = F_GENCODE_STR
gtf.gr = rtracklayer::import(gtf.file) # creates a GRanges object
gtf.df = as.data.frame(gtf.gr)

genes = unique(gtf.df[ ,c("gene_id", "gene_name", "gene_type")])
print(dim(genes))
head(genes)

[1] 55401     3
                gene_id     gene_name            gene_type
1  ENSMUSG00000102693.1 4933401J01Rik                  TEC
4  ENSMUSG00000064842.1       Gm26206                snRNA
7  ENSMUSG00000051951.5          Xkr4       protein_coding
19 ENSMUSG00000102851.1       Gm18956 processed_pseudogene
22 ENSMUSG00000103377.1       Gm37180                  TEC
25 ENSMUSG00000104017.1       Gm37363                  TEC


In [7]:
flipon_list = [
    str(D_FLIPONS_BED / 'mm10.actb_ssdna_enriched_g4.bed'),
    str(D_FLIPONS_BED / 'mm10.actb_ssdna_enriched_sidd.bed'),
    str(D_FLIPONS_BED / 'mm10.actb_ssdna_enriched_z-dna.bed'),
    str(D_FLIPONS_BED / 'mm10.actb_ssdna_enriched_h-dna.bed'),
]
flipon_list

['/home/fpavlov/projects/article_conserved_miRNA/data/flipons_experimental_bed/mm10.actb_ssdna_enriched_g4.bed',
 '/home/fpavlov/projects/article_conserved_miRNA/data/flipons_experimental_bed/mm10.actb_ssdna_enriched_sidd.bed',
 '/home/fpavlov/projects/article_conserved_miRNA/data/flipons_experimental_bed/mm10.actb_ssdna_enriched_z-dna.bed',
 '/home/fpavlov/projects/article_conserved_miRNA/data/flipons_experimental_bed/mm10.actb_ssdna_enriched_h-dna.bed']

In [8]:
%%R -i flipon_list

df_final <- data.frame()

for (flipon in flipon_list) {
  peakAnno <- annotatePeak(toGRanges(flipon), tssRegion=c(-3000, 3000), level='transcript', TxDb=txdb_all_annotation, verbose=FALSE)

  anno_df <- data.frame(peakAnno@anno)
  anno_df$group <- str_replace(str_split(flipon, 'enriched_')[[1]][2], '.bed', '')
  # res_df <- anno_df[grepl("Promoter", anno_df$annotation),][c('annotation', 'geneId')]
  # res_unique_genes_df <- distinct(res_df, geneId, .keep_all=TRUE)

  df_final <- bind_rows(df_final, anno_df)
  
  # print('--------------------------------------------')
  print(flipon)
  # print('--------------------------------------------')
  # print(peakAnno)
  # print(dim(anno_df))
}

# df_final <- merge(df_final, genes, by.x='geneId', by.y='gene_id', all.x=TRUE)
# write.table(df_final, 'geneIds_by_group.tsv', append=FALSE, sep="\t", dec=".", row.names=FALSE, col.names = TRUE)
# df_final


[1] "/home/fpavlov/projects/article_conserved_miRNA/data/flipons_experimental_bed/mm10.actb_ssdna_enriched_g4.bed"
[1] "/home/fpavlov/projects/article_conserved_miRNA/data/flipons_experimental_bed/mm10.actb_ssdna_enriched_sidd.bed"
[1] "/home/fpavlov/projects/article_conserved_miRNA/data/flipons_experimental_bed/mm10.actb_ssdna_enriched_z-dna.bed"
[1] "/home/fpavlov/projects/article_conserved_miRNA/data/flipons_experimental_bed/mm10.actb_ssdna_enriched_h-dna.bed"


In [18]:
df_final = %R df_final
genes = %R genes

flipon_to_gene = df_final.merge(genes, left_on='geneId', right_on='gene_id', how='left')

flipon_to_gene = flipon_to_gene.sort_values(["group", "seqnames", "start"]).assign(
    annotation=lambda df: df["annotation"].apply(
        lambda x: x.split(" ")[0].strip() if "Intron" in x or "Exon" in x else x
    ),
    geneStrand=lambda df: df["geneStrand"].apply(lambda x: "+" if x == 1 else "-"),
    geneChr = lambda df: ('chr' + df['geneChr'].astype(str)).replace('chr20', 'chrX').replace('chr21','chrY'),
    # seqnames = lambda df: df['seqnames'].astype(str).replace('chr20', 'chrX').replace('chr21','chrY'),
    # Coordinates = lambda df: df['name']+':'+df['start'].astype(str)+'-'+df['end'].astype(str),
)[["group", "name", "annotation", "gene_name", "geneStrand", 'gene_type', 'geneId', 'transcriptId']]

flipon_to_gene.columns = [
    "Flipon",
    "Coordinates",
    "Gene Feature",
    "Gene Name",
    "Gene Strand",
    "Gene Type",
    "Gene ID",
    "Transcript ID",
]

flipon_to_gene.to_csv(F_FLIPON_TO_GENE, sep='\t', index=False)
display(flipon_to_gene)


,Flipon,Coordinates,Gene Feature,Gene Name,Gene Strand,Gene Type,Gene ID,Transcript ID
0,g4,chr1:3014794-3014871,Distal Intergenic,4933401J01Rik,+,TEC,ENSMUSG00000102693.1,ENSMUST00000193812.1
1,g4,chr1:3099888-3099963,Promoter (2-3kb),Gm26206,+,snRNA,ENSMUSG00000064842.1,ENSMUST00000082908.1
2,g4,chr1:3287445-3287468,Intron,Gm18956,+,processed_pseudogene,ENSMUSG00000102851.1,ENSMUST00000192857.1
3,g4,chr1:3472953-3472969,Intron,Gm37686,-,TEC,ENSMUSG00000103025.1,ENSMUST00000194099.1
4,g4,chr1:3535948-3535996,Intron,Gm7341,+,processed_pseudogene,ENSMUSG00000103147.1,ENSMUST00000192183.1
...,...,...,...,...,...,...,...,...
60603,z-dna,chrY:3866251-3866287,Distal Intergenic,Gm8521,+,unprocessed_pseudogene,ENSMUSG00000099838.1,ENSMUST00000190394.1
60604,z-dna,chrY:4195990-4196008,Distal Intergenic,Gm29038,-,unprocessed_pseudogene,ENSMUSG00000101108.1,ENSMUST00000191543.1
60605,z-dna,chrY:4202845-4202862,Distal Intergenic,Gm28191,+,unprocessed_pseudogene,ENSMUSG00000100300.1,ENSMUST00000189112.1
60606,z-dna,chrY:4203100-4203146,Distal Intergenic,Gm28191,+,unprocessed_pseudogene,ENSMUSG00000100300.1,ENSMUST00000189112.1
